# 人人汽车推荐系统调研综述
## 这里是一个推荐引擎，使用经典数据集movielens，可以将movies数据替换为人人的车型数据，rating数据替换为从日志系统中收集的所有用户对车的点击次数，浏览时间（权重）。这样可以实现C端对车型的推荐
## 架构： 
### ①日志系统：搜集用户行为提供离线数据  
### ②推荐引擎：A:从数据库或者缓存中拿到用户特征向量（浏览记录 收藏记录 购买记录 停留时间）使用权重最后归一化。B：将用户特征向量通过特征-物品矩阵通过各种推荐算法转换为初始推荐物品列表。C：对初始推荐列表过滤、排名（热门程度、新鲜程度、购买过 去重）。 
### ③UI展示系统 提供标题、缩略图、推荐理由（有了推荐理由用户才愿意点击）评分 根据用户点击情况或者评分计算准确率和召回率来增删推荐引擎或者调整推荐引擎所占的权重。
## 下面是一个初步的推荐算法引擎，只使用了科学计算包numpy和基于numpy的数据处理的包pandas。基于协同过滤算法（collaborative_filter）， surprise或者 python-recsys等框架提供的SVD、余弦相似度算法和后面可能会考虑使用tensorflow提供的深度学习神经网络开发更多推荐引擎。

In [2]:
import numpy as np
import pandas as pd
import os
# 使用pandas加载csv数据

movies = pd.read_csv(os.path.expanduser("~/ml-latest-small/movies.csv"))
ratings = pd.read_csv(os.path.expanduser("~/ml-latest-small/ratings.csv"))
# 去掉无用的维度
ratings.drop(['timestamp'],axis=1,inplace=True)
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [4]:
# 将movieid替换为moviename
def replace_name(x):
    return movies[movies["movieId"]==x].title.values[0]

ratings.movieId = ratings.movieId.map(replace_name)

In [5]:
ratings.head()

,userId,movieId,rating
0,1,Dangerous Minds (1995),2.5
1,1,Dumbo (1941),3.0
2,1,Sleepers (1996),3.0
3,1,Escape from New York (1981),2.0
4,1,Cinema Paradiso (Nuovo cinema Paradiso) (1989),4.0


In [6]:
# 建立一个透视表
M = ratings.pivot_table(index=['userId'],columns=['movieId'],values='rating')

In [7]:
# 当前维度
M.shape

(671, 9064)

In [8]:
# M是一个非常稀疏的透视表
M

movieId,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##  在_产品-产品协同过滤_中的产品之间的相似性值是通过观察所有对两个产品之间的打分的用户来度量的。
<img src="img/item-item.png"/>

## 对于_用户-产品协同过滤_，用户之间的相似性值是通过观察所有同时被两个用户打分的产品来度量的。
<img src="img/user-item.png"/>

# 核心算法方面使用皮尔逊的R来计算距离
### 两个变量之间的皮尔逊相关系数定义为两个变量之间的协方差和标准差的商
![image.svg](img/pearson.svg)
### 上式定义了总体相关系数，常用希腊小写字母 ρ (rho) 作为代表符号。估算样本的协方差和标准差，可得到样本相关系数(样本皮尔逊系数)，常用英文小写字母 r 代表：
![pearson2.svg](img/pearson2.svg)

In [14]:
# 算法实现
def pearson(s1, s2):
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
#     print(f"s1_c={s1_c}")
#     print(f"s2_c={s2_c}")
    denominator = np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))
    if denominator == 0:
        return 0
    return np.sum(s1_c * s2_c) / denominator

###  算法引擎2可以考虑比较文本相似度的余弦相似性算法，这也是推荐系统常用的算法。其中，打分被看成n维空间中的向量，而相似性是基于这些向量之间的角度进行计算的。可以使用sklearn的pairwise_distances函数来计算余弦相似性。注意，输出范围从0到1，因为打分都是正的。

In [15]:
# 永不妥协   碟中谍2
pearson(M['Erin Brockovich (2000)'],M['Mission: Impossible II (2000)'])
# 永不妥协  指环王
# pearson(M['Erin Brockovich (2000)'],M['Fingers (1978)'])
# 永不妥协 哈利波特与密室
# pearson(M['Erin Brockovich (2000)'],M['Harry Potter and the Chamber of Secrets (2002)'])
# 哈利波特与密室  哈利波特与阿兹卡班的囚徒
# pearson(M['Harry Potter and the Chamber of Secrets (2002)'],M['Harry Potter and the Prisoner of Azkaban (2004)'])



0.070185736110215988

In [16]:
def get_recs(movie_name, M, num):
    reviews = []
    for title in M.columns:
        if title == movie_name:
            continue
        cor = pearson(M[movie_name], M[title])
        if np.isnan(cor):
            continue
        else:
            reviews.append((title, cor))
    reviews.sort(key=lambda tup: tup[1], reverse=True)
    return reviews[:num]

In [22]:
# %%time
recs = get_recs('Clerks (1994)', M, 10)
recs[:10]

[('Go Fish (1994)', 0.3873045237464075),
 ('Chasing Amy (1997)', 0.37484047898675804),
 ('Audrey Rose (1977)', 0.36586597215212963),
 ("Razor's Edge, The (1984)", 0.36586597215212957),
 ('Trekkies (1997)', 0.35608001316844984),
 ('Everything Must Go (2010)', 0.34925991438868215),
 ('Out of Sight (1998)', 0.33809493392702311),
 ('Haunting, The (1963)', 0.33177831773334998),
 ('Barefoot in the Park (1967)', 0.33153641267942063),
 ('Flawless (1999)', 0.32978207127880155)]

In [23]:
# %%time
anti_recs = get_recs('Clerks (1994)', M, 8551)
anti_recs[-10:]

[('Enron: The Smartest Guys in the Room (2005)', -0.22634116531793105),
 ('Shot in the Dark, A (1964)', -0.22997876750526428),
 ('Boyhood (2014)', -0.23082944937869748),
 ('Reader, The (2008)', -0.23134059588427391),
 ('Two for the Road (1967)', -0.23590096015604814),
 ('Catfish (2010)', -0.23715890580986979),
 ('Fearless (1993)', -0.23958739682108973),
 ('Lenny (1974)', -0.24222400168825048),
 ('A Home at the End of the World (2004)', -0.2438543994182934),
 ('My Winnipeg (2007)', -0.2438543994182934)]

## 初步设想由大数据同学准备好数据之后，搭建服务进行机器学习，针对每一个车型跑一编结果，缓存起来用户浏览时提供。但是缺陷就是实时性，后面要加入时间上下文因素。
## 但是根据阿里技术专家郑重（卢梭）所说：推荐系统的搭建是个复杂工程，涉及到实时计算、离线计算，以及各种数据采集、流转等，对自建推荐系统来说，1人年是跑不掉的。